In [77]:
import urllib2
import json
import numpy as np
import re
import pandas as pd
import csv
import matplotlib.pyplot as plt 
import collections
from scipy import stats
%matplotlib inline


In [204]:
# 'http://api.census.gov/data/2014/acs5/profile/variables.json'  is a link to the json file showing the codes for variables
def get_variable_list(YYYY):
    url_code = 'http://api.census.gov/data/' + str(YYYY) + '/acs5/profile/variables.json' 
    json_obj=urllib2.urlopen(url_code)
    var_list = json.load(json_obj)
    return var_list

In [219]:
#  http://api.census.gov/data/2015/acs5/variables.json
def get_variable_list_2(YYYY):
    url_code = 'http://api.census.gov/data/' + str(YYYY) + '/acs5/variables.json' 
    json_obj=urllib2.urlopen(url_code)
    var_list = json.load(json_obj)
    return var_list

In [185]:
#B07411. Median Income by Geographical Mobility in the Past Year for Residence 1 Year Ago in the U.S.
#B25128. Tenure by Age of Householder by Year Householder Moved Into Unit
#B25125. Tenure by Age of Householder by Units in Structure
#B25129. Tenure by Year Householder Moved Into Unit by Units in Structure
#B19001. Household Income
#B25036. TENURE BY YEAR STRUCTURE BUILT
#B25034. Year Structure Built
#B07004B. Geographical Mobility in the Past Year (Black or African American Alone) in the U.S.

In [401]:
var_list = get_variable_list(2015)
var_list_2 = get_variable_list_2(2009)

In [402]:
def var_data(var_list, variable_code):
    variable_dic = {}
    variable_list = []
    for code in var_list['variables']:
        if code.startswith(variable_code) and code.endswith('E'):
            variable_list.append(code)
            variable_dic[code] = var_list['variables'][code]['label']
    return variable_list, variable_dic

In [428]:
i_l, i_d = var_data(var_list_2, 'B20001')
b_l, b_d = var_data(var_list_2, 'B01001B')

In [411]:
def collecting_data_for_specific_year(YYYY, state_code,state_abb, var_list,var_dic): 
    #load a json file containing all the codes for variables in variable list
    #find the code on this website http://api.census.gov/data/2014/acs5/variables.json
    census_api_key = 'b9a9255e89915659ba885b13a0e1d5138ccb6ddc'
    base_url = ' http://api.census.gov/data/'
    final_url = base_url + str(YYYY) + '/acs5?get=' + ','.join(sorted(var_list[:50])) + '&for=county+subdivision:*&in=state:' + str(state_code)+'&key=' + str(census_api_key)
    print final_url
    json_data = urllib2.urlopen(final_url)
    dataframe = json.load(json_data)
    dataframe = pd.DataFrame(dataframe)
    remaining_var_list = var_list[50:]
    while len(remaining_var_list) > 0:
        final_url = base_url + str(YYYY) + '/acs5?get=' + ','.join(sorted(remaining_var_list[:50])) + '&for=county+subdivision:*&in=state:' + str(state_code) + '&key=' + str(census_api_key)
        json_data = urllib2.urlopen(final_url)
        df = json.load(json_data)
        df= pd.DataFrame(df)
        dataframe = pd.concat([dataframe.iloc[:, :-3], df], axis = 1)
        remaining_var_list = remaining_var_list[50:]
    dataframe.columns = dataframe.iloc[0, :]
    dataframe = dataframe.iloc[2:, :]
    
    #replace codes with labels for columns
    for code in dataframe.columns[:-3]:
        dataframe.rename(columns={code: var_dic[code]}, inplace=True)
    #find code that matches county name
    county_dic ={}
    county_sub_div_dic={}
    county = 'http://www2.census.gov/geo/docs/reference/codes/files/st' + str(state_code) + '_' + state_abb + '_cousub.txt'
    county = urllib2.urlopen(county).readlines()
    county = csv.reader(county, delimiter = ',') 
    for line in county:
        county_dic[line[2]] = line[3]
        county_sub_div_dic[line[4]] = line[5]
        
    #remove town or Town city from city name
    for key in county_sub_div_dic:
        county_sub_div_dic[key] = re.sub(' t*T*own\s*\w*', '', county_sub_div_dic[key])
        county_sub_div_dic[key] = re.sub(' city', '', county_sub_div_dic[key])

    dataframe['county'] = dataframe['county'].map(county_dic)
    dataframe['county subdivision'] = dataframe['county subdivision'].map(county_sub_div_dic)
    dataframe.set_index(dataframe['county subdivision'], inplace = True)
    return dataframe

In [412]:
def collecting_data_for_multiple_years(years, state_code, state_abb, var_list, var_dict):
    y = {}
    for year in years:
        df = collecting_data_for_specific_year(year, state_code,state_abb, var_list,var_dict)
        y[year] = df
    return y

In [413]:
def percentage_of_population(dic_of_df):
    percentage = {}
    for y in dic_of_df:
        df = dic_of_df[y]
        pop = df.iloc[:, :-3]
        pop = pop.astype(int)
        pop.rename(columns = {'Total:' : 'Total'}, inplace = True)
        pop = pop.div(pop.Total, axis = 0)
        percentage[y] = pop
    return percentage

In [414]:
all_df = collecting_data_for_multiple_years(range(2009,2016), 25, 'ma', i_l, i_d)

 http://api.census.gov/data/2009/acs5?get=B20001_001E,B20001_002E,B20001_003E,B20001_004E,B20001_005E,B20001_006E,B20001_007E,B20001_008E,B20001_009E,B20001_010E,B20001_011E,B20001_012E,B20001_013E,B20001_014E,B20001_015E,B20001_016E,B20001_017E,B20001_018E,B20001_019E,B20001_020E,B20001_021E,B20001_022E,B20001_023E,B20001_024E,B20001_025E,B20001_026E,B20001_027E,B20001_028E,B20001_029E,B20001_030E,B20001_031E,B20001_032E,B20001_033E,B20001_034E,B20001_035E,B20001_036E,B20001_037E,B20001_038E,B20001_039E,B20001_040E,B20001_041E,B20001_042E,B20001_043E&for=county+subdivision:*&in=state:25&key=b9a9255e89915659ba885b13a0e1d5138ccb6ddc
 http://api.census.gov/data/2010/acs5?get=B20001_001E,B20001_002E,B20001_003E,B20001_004E,B20001_005E,B20001_006E,B20001_007E,B20001_008E,B20001_009E,B20001_010E,B20001_011E,B20001_012E,B20001_013E,B20001_014E,B20001_015E,B20001_016E,B20001_017E,B20001_018E,B20001_019E,B20001_020E,B20001_021E,B20001_022E,B20001_023E,B20001_024E,B20001_025E,B20001_026E,B20001

In [415]:
all_df[2010].head()

,Total:,Male:,"Male:!!$1 to $2,499 or loss","Male:!!$2,500 to $4,999","Male:!!$5,000 to $7,499","Male:!!$7,500 to $9,999","Male:!!$10,000 to $12,499","Male:!!$12,500 to $14,999","Male:!!$15,000 to $17,499","Male:!!$17,500 to $19,999",...,"Female:!!$40,000 to $44,999","Female:!!$45,000 to $49,999","Female:!!$50,000 to $54,999","Female:!!$55,000 to $64,999","Female:!!$65,000 to $74,999","Female:!!$75,000 to $99,999","Female:!!$100,000 or more",state,county,county subdivision
county subdivision,,,,,,,,,,,,,,,,,,,,,
Barnstable,25862,13686,639,526,529,285,332,237,290,401,...,763,608,513,622,335,676,416,25,Barnstable County,Barnstable
Bourne,11252,6197,319,375,441,182,247,171,267,143,...,276,246,340,220,230,227,110,25,Barnstable County,Bourne
Brewster,5469,2655,181,154,60,46,121,6,166,74,...,155,189,137,128,119,162,77,25,Barnstable County,Brewster
Chatham,3345,1774,47,180,82,63,84,35,16,47,...,62,34,71,118,86,82,22,25,Barnstable County,Chatham
Dennis,7206,3584,206,92,149,45,233,71,174,104,...,178,153,155,177,178,150,80,25,Barnstable County,Dennis


In [416]:
p_all_df = percentage_of_population(all_df)

In [417]:
p_all_df[2010].head()

,Total,Male:,"Male:!!$1 to $2,499 or loss","Male:!!$2,500 to $4,999","Male:!!$5,000 to $7,499","Male:!!$7,500 to $9,999","Male:!!$10,000 to $12,499","Male:!!$12,500 to $14,999","Male:!!$15,000 to $17,499","Male:!!$17,500 to $19,999",...,"Female:!!$25,000 to $29,999","Female:!!$30,000 to $34,999","Female:!!$35,000 to $39,999","Female:!!$40,000 to $44,999","Female:!!$45,000 to $49,999","Female:!!$50,000 to $54,999","Female:!!$55,000 to $64,999","Female:!!$65,000 to $74,999","Female:!!$75,000 to $99,999","Female:!!$100,000 or more"
county subdivision,,,,,,,,,,,,,,,,,,,,,
Barnstable,1.0,0.529193,0.024708,0.020339,0.020455,0.011020,0.012837,0.009164,0.011213,0.015505,...,0.025713,0.042881,0.029000,0.029503,0.023509,0.019836,0.024051,0.012953,0.026139,0.016085
Bourne,1.0,0.550747,0.028351,0.033327,0.039193,0.016175,0.021952,0.015197,0.023729,0.012709,...,0.019552,0.023196,0.033772,0.024529,0.021863,0.030217,0.019552,0.020441,0.020174,0.009776
Brewster,1.0,0.485464,0.033096,0.028159,0.010971,0.008411,0.022125,0.001097,0.030353,0.013531,...,0.039678,0.048455,0.051563,0.028342,0.034558,0.025050,0.023405,0.021759,0.029622,0.014079
Chatham,1.0,0.530344,0.014051,0.053812,0.024514,0.018834,0.025112,0.010463,0.004783,0.014051,...,0.028700,0.053513,0.013453,0.018535,0.010164,0.021226,0.035277,0.025710,0.024514,0.006577
Dennis,1.0,0.497363,0.028587,0.012767,0.020677,0.006245,0.032334,0.009853,0.024147,0.014432,...,0.046489,0.035942,0.033167,0.024702,0.021232,0.021510,0.024563,0.024702,0.020816,0.011102


In [418]:
z = pd.read_csv('City_MedianSoldPricePerSqft_AllHomes.csv')
t = []
for i in z.iloc[:, 6:].columns.tolist():
    t.append(i[:-3])
z_ma = z.loc[z['State'] == 'MA'].set_index(['RegionName']).iloc[:, 5:]
z_ma.columns = t
z_ma = z_ma.transpose().groupby(t).mean()



In [423]:
def common_cities_between_zillow_and_df(dic_of_df, zillow):
    df_w_zillow ={}
    for y in dic_of_df:
        df = dic_of_df[y]
        df['price'] = 0
        for cities in df.index.tolist():
            try:
                df.loc[cities, 'price'] = zillow.loc[str(y), cities]
            except (KeyError, TypeError):
                continue
        df = df.loc[df['price'] > 0]
        df_w_zillow[y] = df
    return df_w_zillow

In [424]:
price_all_df = common_cities_between_zillow_and_df(p_all_df, z_ma)
price_all_df[2010].head()

,Total,Male:,"Male:!!$1 to $2,499 or loss","Male:!!$2,500 to $4,999","Male:!!$5,000 to $7,499","Male:!!$7,500 to $9,999","Male:!!$10,000 to $12,499","Male:!!$12,500 to $14,999","Male:!!$15,000 to $17,499","Male:!!$17,500 to $19,999",...,"Female:!!$30,000 to $34,999","Female:!!$35,000 to $39,999","Female:!!$40,000 to $44,999","Female:!!$45,000 to $49,999","Female:!!$50,000 to $54,999","Female:!!$55,000 to $64,999","Female:!!$65,000 to $74,999","Female:!!$75,000 to $99,999","Female:!!$100,000 or more",price
county subdivision,,,,,,,,,,,,,,,,,,,,,
Chatham,1.0,0.530344,0.014051,0.053812,0.024514,0.018834,0.025112,0.010463,0.004783,0.014051,...,0.053513,0.013453,0.018535,0.010164,0.021226,0.035277,0.025710,0.024514,0.006577,337.102958
Dennis,1.0,0.497363,0.028587,0.012767,0.020677,0.006245,0.032334,0.009853,0.024147,0.014432,...,0.035942,0.033167,0.024702,0.021232,0.021510,0.024563,0.024702,0.020816,0.011102,307.291667
Eastham,1.0,0.540221,0.031643,0.019825,0.024400,0.017156,0.033168,0.017537,0.029356,0.013725,...,0.059093,0.010294,0.038506,0.004575,0.021731,0.020587,0.007625,0.025543,0.004956,268.606070
Harwich,1.0,0.495534,0.031426,0.018028,0.026464,0.006285,0.013397,0.006947,0.010751,0.014555,...,0.038538,0.037380,0.023983,0.038042,0.024975,0.039034,0.005624,0.019848,0.023487,226.181886
Mashpee,1.0,0.504435,0.023480,0.018784,0.018262,0.020089,0.025046,0.004305,0.015393,0.002870,...,0.036655,0.036003,0.027785,0.017088,0.024393,0.020480,0.017219,0.024393,0.012784,206.311942


In [425]:
important_income =[]
for year in price_all_df:
    df = price_all_df[year]
    x_var = df.iloc[:, :-1]
    y = df['price'].tolist()
    for income in x_var:
        x = x_var[income].tolist()
        slope,intercept,r_value,p_value,std_err = stats.linregress(x,y)
        if r_value**2 > 0.3:
            important_income.append((year, income, r_value**2))
important_income

[(2010, u'Female:!!$100,000 or more', 0.39392297919265029),
 (2011, u'Female:!!$100,000 or more', 0.41474430295743675),
 (2012, u'Female:!!$100,000 or more', 0.37447187563821499),
 (2013, u'Female:!!$100,000 or more', 0.38233022244973641),
 (2014, u'Female:!!$100,000 or more', 0.40544989557227007),
 (2015, u'Male:!!$50,000 to $54,999', 0.31416127282322998),
 (2015, u'Female:!!$25,000 to $29,999', 0.37611939355858059),
 (2015, u'Female:!!$100,000 or more', 0.44592347152446421)]